In [2]:
import os
import pandas as pd

In [27]:
# Directories for each method
embeddings = ['w2v', 'dnabert1-pretrained', 'dnabert1-finetuned-motifs', 'dnabert2-pretrained', 'dnabert2-finetuned-cancer', 'grover-finetuned-cancer']

splits = ['split1', 'split2', 'split3']

results_df = pd.DataFrame(columns=['split', 'embedding', 'embedding_args', 'learning_rate', 'num_epochs', 'hidden_size', 'acc', 'f1-score', 'mcc'])
# Dictionary to store all data for each method
all_data = {split: {} for split in splits}

# Dictionary to store the best parameters and metrics for each method
best_metrics = {split: {} for split in splits}

# Path to the main directory containing method directories
base_path = 'results-novo/'

idx = 0
# Iterate through each method
for embedding in embeddings:

    # model_n = 1

    for split in splits:

        best_acc = 0
        best_f1 = 0
        best_mcc = 0

        split_results = []
        
        method_path = os.path.join(base_path, embedding, split)
        # print(method_path)
        
        
        # Iterate through each numbered subdirectory in the method directory
        for sub_dir in os.listdir(method_path):
        # sub_dir_path = os.path.join(method_path, sub_dir)
            
        #     if os.path.isdir(sub_dir_path):
            log_path = os.path.join(method_path, sub_dir, 'log.out')
            # print(log_path)

            # model_n += 1
                
                # Check if log.out exists
            if os.path.exists(log_path):
                with open(log_path, 'r') as log_file:
                    lines = log_file.readlines()

                    # print(lines)
                        
                    # Extract paramete  rs and metrics from the log file
                    embedding_args = ''.join(lines[3].split(': ')[2:])
                    embedding_args = embedding_args[0:-2].strip()
                    # print(embedding_args)
                    learning_rate = float(lines[6].split(': ')[1].strip())
                    num_epochs = int(lines[7].split(': ')[1].strip())
                    hidden_size = int(lines[8].split(': ')[1].strip())
                    acc = round(float(lines[15].split(': ')[1].strip()),2)
                    f1 = round(float(lines[18].split(': ')[1].strip()),2)
                    mcc = round(float(lines[19].split(': ')[1].strip()),2)
                    
                    # Save data
                    data = [
                        int(split[-1]),
                        embedding,
                        embedding_args,
                        learning_rate,
                        num_epochs,
                        hidden_size,
                        acc,            
                        f1,
                        mcc
                    ]

                    # print(embedding_args)
                    # print(data)
                    results_df.loc[idx] = data
                    idx += 1

                #     if acc > best_metrics[split]['acc'] and f1 > best_metrics[embedding]['f1']:
                #         best_metrics[embedding]['acc'] = acc
                #         best_metrics[embedding]['f1'] = f1
                #         best_metrics[embedding]['mcc'] = mcc
                #         data_no_metrics = updated_dict = {k: v for k, v in data.items() if k != 'acc' and k != 'f1' and k != 'mcc'}
                #         best_metrics[embedding]['params'] = data_no_metrics


                #     split_results.append(data)
                        
                # all_data[split][embedding] = split_results

In [28]:
results_df

,split,embedding,embedding_args,learning_rate,num_epochs,hidden_size,acc,f1-score,mcc
0,1,w2v,'./w2v-models/50_10_100_250_model',0.0003,20,250,0.63,0.63,0.26
1,1,w2v,'./w2v-models/50_5_100_250_model',0.0030,100,250,0.62,0.62,0.25
2,1,w2v,'./w2v-models/50_10_250_250_model',0.0003,20,250,0.63,0.63,0.25
3,1,w2v,'./w2v-models/100_10_250_250_model',0.0003,100,250,0.61,0.61,0.23
4,1,w2v,'./w2v-models/50_10_250_250_model',0.0003,100,250,0.63,0.63,0.26
...,...,...,...,...,...,...,...,...,...
163,3,grover,'mean',0.0030,20,768,0.70,0.70,0.40
164,3,grover,'mean',0.0003,20,768,0.72,0.72,0.44
165,3,grover,'max',0.0003,20,768,0.71,0.71,0.42
166,3,grover,'max',0.0030,20,768,0.68,0.68,0.37


In [29]:
results_df.columns

Index(['split', 'embedding', 'embedding_args', 'learning_rate', 'num_epochs',
       'hidden_size', 'acc', 'f1-score', 'mcc'],
      dtype='object')

In [48]:
split_metrics = results_df.groupby(['split', 'embedding'])[['acc', 'f1-score', 'mcc']].mean()
split_metrics = split_metrics.round(3)

In [42]:
split_metrics

acc  f1-score   mcc
split embedding                      
1     dnabert1   0.50      0.33  0.00
      dnabert2   0.69      0.68  0.40
      grover     0.66      0.62  0.34
      w2v        0.62      0.62  0.24
2     dnabert1   0.50      0.33  0.00
      dnabert2   0.69      0.68  0.39
      grover     0.69      0.69  0.40
      w2v        0.62      0.61  0.25
3     dnabert1   0.50      0.33  0.00
      dnabert2   0.69      0.68  0.38
      grover     0.70      0.70  0.41
      w2v        0.61      0.61  0.23

In [49]:
embedding_metrics = split_metrics.groupby(['embedding'])[['acc', 'f1-score', 'mcc']].mean()
embedding_metrics = embedding_metrics.round(3)

In [50]:
embedding_metrics

,acc,f1-score,mcc
embedding,,,
dnabert1,0.500,0.330,0.000
dnabert2,0.689,0.684,0.389
grover,0.685,0.671,0.384
w2v,0.616,0.612,0.238


In [7]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


NameError: name 'learning_rate' is not defined